<a href="https://colab.research.google.com/github/ayushkalani/logic-dataset-creation-nlp/blob/master/Multiple_choice_on_QUAIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [54]:
!pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
import transformers

print(transformers.__version__)

4.12.5


If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

# Fine-tuning a model on a multiple choice task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a multiple choice task, which is the task of selecting the most plausible inputs in a given selection. The dataset used here is [SWAG](https://www.aclweb.org/anthology/D18-1009/) but you can adapt the pre-processing to any other multiple choice dataset you like, or your own data. SWAG is a dataset about commonsense reasoning, where each example describes a situation then proposes four options that could go after it. 

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a mutiple choice head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those two parameters, then the rest of the notebook should run smoothly:

In [3]:
model_checkpoint = "bert-base-uncased"
batch_size = 4

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the functions `load_dataset`.  

In [4]:
from datasets import load_dataset, load_metric

`load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [180]:
datasets = load_dataset("quail")

Reusing dataset quail (/root/.cache/huggingface/datasets/quail/quail/1.3.0/3cabab19c99e571b528209e14313cfff1debf772db9e24e19b4fcbeb8399336c)


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [126]:
datasets
datasets = datasets.rename_column("correct_answer_id", "label")

To access an actual element, you need to select a split first, then give an index:

In [181]:
datasets["train"][0]

{'answers': ['not enough information',
  'to visit family',
  'parents had problems',
  'for tutoring'],
 'context': "That fall came and I went back to Michigan and the school year went by and summer came and I never really thought about it. I'm not even sure if I was officially asked, I just wound up heading back to New Jersey when school was out. I think my parents thought it was a good enough deal. They were already having some problems and without Nonna there anymore to take care of me I think my cousin's house on the coast seemed like as good a spot as any to stick me for the summer. It certainly wasn't because of any great love between me and my cousin. We weren't really very good friends at that point. I think she saw me as sort of foisted off on her and getting in the way of her summers. Which was a fair enough judgment. But she could have been nicer. It's pretty amazing that she wound up as my Maid of Honor. Time does strange things. Your lovable jack-ass of a father would men

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

Each example in the dataset has a context composed of a first sentence (in the field `sent1`) and an introduction to the second sentence (in the field `sent2`). Then four possible endings are given (in the fields `ending0`, `ending1`, `ending2` and `ending3`) and the model must pick the right one (indicated in the field `label`). The following function lets us visualize a give example a bit better:

In [10]:
def show_one(example):
    print(f"Context: {example['sent1']}")
    print(f"  A - {example['sent2']} {example['ending0']}")
    print(f"  B - {example['sent2']} {example['ending1']}")
    print(f"  C - {example['sent2']} {example['ending2']}")
    print(f"  D - {example['sent2']} {example['ending3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [12]:
# show_one(datasets["train"][0])

In [13]:
# show_one(datasets["train"][15])

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [14]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [15]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

We can them write the function that will preprocess our samples. The tricky part is to put all the possible pairs of sentences in two big lists before passing them to the tokenizer, then un-flatten the result so that each example has four input ids, attentions masks, etc.

When calling the `tokenizer`, we use the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [186]:
# ending_names = ["ending0", "ending1", "ending2", "ending3"]

# def preprocess_function(examples):
#     # Repeat each first sentence four times to go with the four possibilities of second sentences.
#     first_sentences = [[context] * 4 for context in examples["sent1"]]
#     # Grab all second sentences possible for each context.
#     question_headers = examples["sent2"]
#     second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
#     # Flatten everything
#     first_sentences = sum(first_sentences, [])
#     second_sentences = sum(second_sentences, [])
    
#     # Tokenize
#     tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
#     # Un-flatten
#     return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}
context_name = "context"
question_header_name = "question"
answers_key_name = "answers"

def preprocess_function(examples):
    question_headers = examples[question_header_name]
    first_sentences = []
    for header, context in zip(question_headers, examples[context_name]):
        first_sentences.append([f"{context} {header}"]*4)
    second_sentences = examples[answers_key_name]
    # Flatten out
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    # display(first_sentences)
    # display(second_sentences)

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # # Un-flatten
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (4) and a list of input IDs (length varying here since we did not apply any padding):

In [187]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])
features.keys()

5 4 [381, 381, 381, 381]


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

To check we didn't do anything group when grouping all possibilites then unflattening, let's have a look at the decoded inputs for a given example:

In [135]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

["[CLS] that fall came and i went back to michigan and the school year went by and summer came and i never really thought about it. i'm not even sure if i was officially asked, i just wound up heading back to new jersey when school was out. i think my parents thought it was a good enough deal. they were already having some problems and without nonna there anymore to take care of me i think my cousin's house on the coast seemed like as good a spot as any to stick me for the summer. it certainly wasn't because of any great love between me and my cousin. we weren't really very good friends at that point. i think she saw me as sort of foisted off on her and getting in the way of her summers. which was a fair enough judgment. but she could have been nicer. it's pretty amazing that she wound up as my maid of honor. time does strange things. your lovable jack - ass of a father would mention something about magic in here. you know if you took a group of fifty strangers, had them chat with your

We can compare it to the ground truth:

In [19]:
# show_one(datasets["train"][3])

This seems alright, so we can apply this function on all the examples in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [136]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/quail/quail/1.3.0/3cabab19c99e571b528209e14313cfff1debf772db9e24e19b4fcbeb8399336c/cache-ca66904279ba8bf1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/quail/quail/1.3.0/3cabab19c99e571b528209e14313cfff1debf772db9e24e19b4fcbeb8399336c/cache-f207a2823e4eb0fa.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/quail/quail/1.3.0/3cabab19c99e571b528209e14313cfff1debf772db9e24e19b4fcbeb8399336c/cache-2c213b7b8fb82753.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our task is about mutliple choice, we use the `AutoModelForMultipleChoice` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [21]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [22]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-swag"` or `"huggingface/bert-finetuned-swag"`).

Then we need to tell our `Trainer` how to form batches from the pre-processed inputs. We haven't done any padding yet because we will pad each batch to the maximum length inside the batch (instead of doing so with the maximum length of the whole dataset). This will be the job of the *data collator*. A data collator takes a list of examples and converts them to a batch (by, in our case, applying padding). Since there is no data collator in the library that works on our specific problem, we will write one, adapted from the `DataCollatorWithPadding`:

In [156]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        display(features[0].keys())
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

When called on a list of examples, it will flatten all the inputs/attentions masks etc. in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * 4) x seq_length`) that we then unflatten.

We can check this data collator works on a list of features, we just have to make sure to remove all features that are not inputs accepted by our model (something the `Trainer` will do automatically for us after):

In [164]:
accepted_keys = ["input_ids", "attention_mask", "label"]
display(encoded_datasets["train"][0].items())
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

dict_items([('answers', ['not enough information', 'to visit family', 'parents had problems', 'for tutoring']), ('attention_mask', [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

dict_keys(['attention_mask', 'input_ids', 'label'])

Again, all those flatten/un-flatten are sources of potential errors so let's make another sanity check on our inputs:

In [25]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]

["[CLS] that fall came and i went back to michigan and the school year went by and summer came and i never really thought about it. i'm not even sure if i was officially asked, i just wound up heading back to new jersey when school was out. i think my parents thought it was a good enough deal. they were already having some problems and without nonna there anymore to take care of me i think my cousin's house on the coast seemed like as good a spot as any to stick me for the summer. it certainly wasn't because of any great love between me and my cousin. we weren't really very good friends at that point. i think she saw me as sort of foisted off on her and getting in the way of her summers. which was a fair enough judgment. but she could have been nicer. it's pretty amazing that she wound up as my maid of honor. time does strange things. your lovable jack - ass of a father would mention something about magic in here. you know if you took a group of fifty strangers, had them chat with your

All good!

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits:

In [27]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [60]:
small_train_dataset = encoded_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = encoded_datasets["challenge"].shuffle(seed=42).select(range(100))
full_train_dataset = encoded_datasets["train"]
full_eval_dataset = encoded_datasets["challenge"]
trainer = Trainer(
    model,
    args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/quail/quail/1.3.0/3cabab19c99e571b528209e14313cfff1debf772db9e24e19b4fcbeb8399336c/cache-910cc2baa2ba6d47.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/quail/quail/1.3.0/3cabab19c99e571b528209e14313cfff1debf772db9e24e19b4fcbeb8399336c/cache-52a7aacd6e033061.arrow


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

In [34]:
from google.colab import drive
drive.mount('/content/drive')
!python --version
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/CSE576')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Python 3.7.12
/content/drive/My Drive/CSE572_DataMining
/content/drive/MyDrive/CSE576


In [ ]:
import pandas as pd

df=pd.read_csv('ManualCleanedDataframe.csv')
df['context']=df['context']+df['question']
df['answer']=df['answer'].astype(str)
df = df.rename(columns={'answer':'label'})
df

In [170]:
context_name = "context"
question_header_name = "question"
answers_key_name = "label"

def preprocess_manual(examples):
    first_sentences = list(examples[context_name])
    second_sentences = list(examples[answers_key_name])
    # Flatten out
    # display(first_sentences)
    # display(second_sentences)

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    display(tokenized_examples.items())
    # # Un-flatten
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [173]:
manual_dataset_tokens=preprocess_manual(df)

features = preprocess_manual(df[:10])
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])
[tokenizer.decode(features["input_ids"][0][i]) for i in range(4)]

dict_items([('input_ids', [[101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 2022, 2593, 2652, 2374, 2030, 10424, 2483, 11306, 1012, 2572, 1045, 2012, 1996, 3509, 2065, 10047, 2652, 10424, 2483, 11306, 1029, 102, 2995, 102], [101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 2022, 2593, 2652, 2374, 2030, 10424, 2483, 11306, 1012, 2572, 1045, 2012, 1996, 2380, 2065, 10047, 2652, 4715, 1029, 102, 2995, 102], [101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 2

dict_items([('input_ids', [[101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 2022, 2593, 2652, 2374, 2030, 10424, 2483, 11306, 1012, 2572, 1045, 2012, 1996, 3509, 2065, 10047, 2652, 10424, 2483, 11306, 1029, 102, 2995, 102], [101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 2022, 2593, 2652, 2374, 2030, 10424, 2483, 11306, 1012, 2572, 1045, 2012, 1996, 2380, 2065, 10047, 2652, 4715, 1029, 102, 2995, 102], [101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 2

3 4 [62, 60, 61, 61]


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [169]:
accepted_keys = ["input_ids", "attention_mask", "label"]
display(manual_dataset_tokens.items())
display(encoded_datasets["train"][0].items())
features = [{k: v for k, v in manual_dataset_tokens.items() if k in accepted_keys}]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

dict_items([('input_ids', [[[101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 2022, 2593, 2652, 2374, 2030, 10424, 2483, 11306, 1012, 2572, 1045, 2012, 1996, 3509, 2065, 10047, 2652, 10424, 2483, 11306, 1029, 102, 2995, 102], [101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 2022, 2593, 2652, 2374, 2030, 10424, 2483, 11306, 1012, 2572, 1045, 2012, 1996, 2380, 2065, 10047, 2652, 4715, 1029, 102, 2995, 102], [101, 2065, 2175, 2000, 1996, 3509, 1010, 1045, 2377, 10424, 2483, 11306, 1012, 2065, 2175, 2000, 1996, 2380, 1010, 1045, 2377, 4715, 1012, 1045, 2572, 2593, 2012, 1996, 3509, 2030, 2012, 1996, 2380, 1012, 3568, 1045, 1005, 2222, 

dict_items([('answers', ['not enough information', 'to visit family', 'parents had problems', 'for tutoring']), ('attention_mask', [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

dict_keys(['input_ids', 'attention_mask'])

KeyError: ignored

In [91]:
manual_trainer = Trainer(
    model,
    args,
    train_dataset=manual_dataset_token,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

In [92]:
manual_trainer.train()

***** Running training *****
  Num examples = 3
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3


KeyError: ignored